# [Promises for asynchronous programming](https://exploringjs.com/impatient-js/ch_promises.html)

## The basics of using Promises

* promises are a technique for delivering results asynchronously

### Using a Promise-based function

* Promises are similar to the event pattern
    - there is an object (Promise) where we register callbacks:
        * Method .then() registers callbacks that handles results
        * Method .catch() registers callbacks that handle errors
* a Promise-based function returns a Promise and sends it a result or an error (if and when it is done)
    - the promise passes it on to the relevant callbacks
* in contrast to the event pattern, Promises are optimized for one-off results:
    - a result (or an error) is cached so that it doesn't matter if we register a callback before or after the result (or error) was sent
    - .then() and .catch() can both be chained b/c they return Promises
        * helps with sequentially invoking multiple asynchronous functions

In [3]:
Promise-based function
addAsync(3, 4)
    .then(result => { // success
        assert.equal(result, 7);
    })
    .catch(error => { // failure
        assert.fail(error);
    })

const addPositiveAsync = (x, y) => {
    return new Promise((resolve, reject) => {
      if (x < 0 || y < 0) {
        reject('X and Y should be positive numbers!');
    }
    else {
        setTimeout(() => {
          resolve(x + y);
      }, 1000)
    }
  });
}

const callAsync = async () => {
    try {
        let result = await addPositiveAsync(-1, -2);
    console.log(result);
  }
  catch(error) {
      console.error(error);
  }
  
  try {
      let result = await addPositiveAsync(1, 2);
        console.log(result);
  }
  catch(error) {
      console.error(error);
  }
}

callAsync();

Promise { <pending> }

X and Y should be positive numbers!


3


### What is a Promise?

* two ways of looking at it:
    1. placeholder or container for the final result that will eventually be delivered
    2. an object with which we can register listeners

### Implementing a Promise-based function

* addAsync() immediately invokes the Promise constructor
* the implementation of the function resides in the callback that is passed to the constructor (line A)
* the callback is provided with 2 functions:
    1. resolve is used for delivering a result (in case of success)
    2. reject is used for delivering an error (in case of failure)

In [ ]:
// implementation of a Promise-based function that adds two numbers x and y:
function addAsync(x, y) {
    return new Promise(
        (resolve, reject) => { // A
            if (x === undefined || y === undefined) {
                reject(new Error('Must provide two parameters'));
            }
            else {
                resolve(x + y);
            }
        }
    )
}

### States of Promises

![States of Promises](https://exploringjs.com/impatient-js/img-book/promises/promise_states_simple.svg)

* a Promise can be in one of three states:
    1. pending
    2. fulfilled
    3. rejected
* if a Promise is in a final (non-pending) state, it is called settled
* promises specialize in one-off results and protect us against race conditions
    - race conditions = registering too early or too late
    - if we register .then() or .catch() callback too early, it is notified once a Promise is settled
    - once a Promise is settled, the settlement value (result or error) is cached
        * so .then() or .catch() are called after the settlement, they receive the cached value
* once a Promise is settled, its state and settlemenet value can't change anymore
    - this helps make the code predictable and enforces the one-off nature of Promises

### Promise.resolve(): create a Promise fulfilled with a given value

In [ ]:
Promise.resolve(123)
    .then(x => {
        assert.equal(x, 123);
    })

In [ ]:
// if the parameter is already a Promise, it is returned unchanged
// so, given ab arbitrary value x, we can use Promise.resolve(x) to ensure we have a Promise
// the name is resolve, and not fulfill, b/c .resolve() returns a rejected Promise if its Parameter is a rejected Promise

const abcPromise = Promise.resolve('abc');
assert.equal(
    Promise.resolve(abcPromise),
    abcPromise
)

### Promise.reject() create a Promise rejected with a given value

In [ ]:
const myError = new Error('My error!');
Promise.reject(myError)
    .catch(err => {
        assert.equal(err, myError);
    })

### Returning and throwing in .then() callbacks

* .then() handles Promise fulfillments
* it also returns a fresh Promise
* how that Promise is settled depends on what happens inside the callback and there are three common cases

#### Returning a non-Promise value

* first, the callback can return a non-Promise value (line A)
* the Promise returned by .then() is fulfilled with that value (in line B)

In [ ]:
Promise.resolve('abc')
    .then(str => {
        return str + str; // A
    })
    .then(str2 => {
        assert.equal(str2, 'abcabc'); // B
    })

#### Returning a Promise

* second, the callback can return a Promise p (line A)
* consequently, p "becomes" what .then() returns
    - i.e. the Promise that .then() has already returned is effectively replaced by p
* why is that useful?
    - b/c we can return the result of a Promise-based operation and process its fulfillment value via a "flat" (non-nested) .then()
    - basically, we can just chain .then() methods instead of having them nested inside of 1

In [ ]:
Promise.resolve('abc')
    .then(str => {
        return Promise.resolve(123); // A
    })
    .then(num => {
        assert.equal(num, 123)
    })

In [ ]:
// Flat
asyncFunc1()
    .then(result1 => {
        /*...*/
        return asyncFunc2();
    })
    .then(result2 => {
        /*...*/
    });

// Nested
asyncFunc1()
    .then(result1 => {
        /*...*/
        asyncFunc2()
            .then(result2 => {
                /*...*/
            })
    })

#### Throwing an exception

* third, the callback can throw an exception
* the Promise returned by .then() is rejected with that exception
    - that is, a synchronous error is converted into an asynchronous error


In [ ]:
const myError = new Error('My error!');
Promise.resolve('abc')
    .then(str => {
        throw myError;
    })
    .catch(err => {
        assert.equal(err, myError);
    })

### .catch() and its callback

* the difference between .then() and .catch() is that the latter is triggered by rejections, not fulfillments
    - but both methods rurn the actions of their callbacks into Promises in the same manner
* for example, the value returned by the .catch() callback in line A becomes a fulfillment value

In [ ]:
const err = new Error();

Promise.reject(err)
    .catch(e => {
        assert.equal(e, err);
        // something went wrong, use a default value
        return 'default value'; // A
    })
    .then(str => {
        assert.equal(str, 'default value')
    })

### Chaining method calls

* .then() and .catch() __always return Promises__
* this allows us to create arbitrary long chains of method calls
* due to chaining, the return in line A returns the result of the last .then()
* in a way, .then() is the asynchronous version of the synchronous semicolon
    - .then() executes 2 asynchronous operations sequentially
    - the semicolon executes 2 synchronous operations sequentially

In [ ]:
function myAsyncFunc() {
    return asyncFunc1() // A
        .then(result1 => {
            // ...
            return asyncFunc2(); // a Promise
        })
        .then(result2 => {
            // ...
            return result2 ?? '(Empty)'; // not a Promise
        })
        .then(result3 => {
            // ...
            return asyncFunc4(); // a Promise
        })
}

* we can also add .catch() into the mix and let it handle multiple error sources at the same time

In [ ]:
asyncFunc1()
    .then(result1 => {
        // ...
        return asyncFunction2();
    })
    .then(result2 => {
        // ...
    })
    .catch(error => {
        // Failure: handle errors of asyncFunc1(), asyncFunc2()
        // and any (sync) exceptions thrown in previous callbacks
    })

### .finally()

* the .finally() callback is always executed - independently of somePromise and the values returned by .then() and/or .catch()
* in contrast:
    - the .then() callback is only executed if somePromise is fulfilled
    - the .catch() callback is only executed if:
        * either somePromise is rejected
        * or the .then() callback returns a rejected Promise
        * or the .then() callback throws an exception

In [ ]:
somePromise
    .then((result) => {
        // ...
    })
    .catch((error) => {
        // ...
    })
    .finally(() => {
        // ...
    });

* .finally() ignores what its callback returns and simply passes on the settlement that existed before it was called:

In [ ]:
Promise.resolve(123)
    .finally(() => {})
    .then((result) => {
        assert.equal(result, 123);
    })

Promise.reject('error')
    .finally(() => {})
    .catch((error) => {
        assert.equal(error, 'error')
    })

* if the .finally() callback throws an exception, the Promise returned by .finally() is rejected

In [ ]:
Promise.reject('error (originally)')
    .finally(() => {
        throw 'error (finally)';
    })
    .catch((error) => {
        assert.equal(error, 'error (finally)');
    })

##### Use case for .finally(): cleaning up

* cleaning up after you are done with a resource regardless if it was successful or failed

In [ ]:
let connection;

db.open()
.then((conn) => {
    connection = conn;
    return connection.select({ name: 'Jane '})
})
.then((result) => {
    // Process result
    // Use 'connection' to make more queries
})
//...
.catch((error) => {
    //handle errors
})
.finally(() => {
    connection.close();
});

##### Use case for .finally(): doing something first after any kind of settlement

* can use .finally() before both .then() and .catch()
* .finally() will be executed before the other two callbacks

In [2]:
Promise.resolve('fulfilled')
    .finally(() => {
        console.log('finally');
    })
    .then((result) => {
        console.log('then ' + result);
    })
    .catch((error) => {
        console.log('catch ' + error);
    })

// Output:
// 'finally'
// 'then fulfilled'

Promise.reject('rejected')
    .finally(() => {
        console.log('finally');
    })
    .then((result) => {
        console.log('then ' + result);
    })
    .catch((error) => {
        console.log('catch ' + error);
    })

// Output:
// 'finally'
// 'catch rejected'

Promise { <pending> }

finally
finally
then fulfilled
catch rejected


### Advantages of promises over plain callbacks

* when it comes to handling one-off results:
    - parameters in Promise-based functions and methods are cleaner
        * in a Promise, you only have resolve and reject to worry about as far as parameters go
        * with callbacks, the first couple of arguments are for input to the function while the last arguments are for the actual callback function
            - so determining what is an input parameter and what is a callback function can be confusing
    - chaining asynchronous processing steps is more convenient
    - Promises handle both asynchronous errors (via rejections) and synchronous errors
        * inside the callbacks for new Promise(), .then(), and .catch(), exceptions are converted to rejections
        * if we use callbacks, exceptions are not handled for us
    - Promises are a single standard
* also the biggest advantage of Promises is being able to use the async/await syntax
    - async/await is a synchronous-looking syntax for performing asynchronous computations

## Error handling: don't mix rejections and exceptions

* don't mix (asynchronous) rejections and (synchronous) exceptions
* simplifies our code b/c we only need to focus on one error-handling mechanic
* so for Promise-based functions/methods, they should never throw exceptions

In [ ]:
// DON'T DO THIS:

function asyncFunc() {
    doSomethingSync(); // A
    return doSomethingAsync()
        .then(result => {
        // ...
    })
}

* if doSomethingSync in line A throws an exception, then asyncFunc() will also throw an exception when it should be throwing a rejection
* callers of asyncFunc() are expecting a rejection

* Solution 1:
    - we can wrap the whole body of asyncFunc in a try-catch statement and return a rejected Promise if an exception is thrown

In [ ]:
// Solution 1

function asyncFunc() {
    try {
        doSomethingSync();
        return doSomethingAsync()
            .then(result => {
            // ...
        });
    }
    catch(err) {
        return Promise.reject(err);
    }
}

* Solution 2:
    - we know that .then() will convert exceptions to rejections so we can just call doSomethingSync() in the .then() callback
* we can do this by creating a Promise chain using Promise.resolve() to get us into the .then() callback
    - the fulfillment value, which is undefined, does not matter for Promise.resolve()

In [1]:
// Solution 2

function asyncFunc() {
    return Promise.resolve()
        .then(() => {
            doSomethingSync();
            return doSomethingAsync();
        })
        .then(result => {
            // ...
        });
}

* Solution 3:
    - new Promise() constructor can also convert exceptions to rejections

In [2]:
// Solution 3

function asyncFunc() {
    return new Promise((resolve, reject) => {
        doSomethingSync();
        resolve(doSomethingAsync());
    })
    .then(result => {
        // ...
    })
}

## Promise-based functions start synchronously, settle asynchronously

* Promise-based functions are executed as follows:
    - their execution starts right away, synchronously (in the current task)
    - but the Promise they return is guaranteed to be settled asynchronously (in a later task) - if ever
* in the example, we can see that the callback of new Promise() is executed before the end of the code, while the result is delivered later (line A)
* Benefits of this approach:
    - starting synchronously helps avoid race conditions b/c we can rely on the order in which Promise-based functions begin
    - Chaining Promises won't starve other tasks of processing time b/c before a Promise is settled, there will always be a break, during which the event loop can run
    - Promise-based functions always return results asynchronously; we can be sure that there is never a synchronous return

In [4]:
function asyncFunc() {
    console.log('asyncFunc');
    return new Promise(
        (resolve, _reject) => {
            // this callback is called synchronously
            console.log('new Promise()');
            
            // but resolving the callback happens asynchronously
            resolve();
        });
}

console.log('START');
asyncFunc()
    .then(() => {
        // we need to wait for the new Promise callback to be resolved first
        // before this is console logged
        console.log('.then()'); // (A)
    });
console.log('END')



START
asyncFunc
new Promise()
END
.then()


## Promise combinator functions: working with Arrays of Promises

### What is a Promise combinator function?

* combinator is a pattern in functional programming for building structures using 2 kinds of funcitons:
    - primitive functions (primitives): creates atomic pieces
    - combinator functions (combinators): combine atomic and/or compound pieces to create compound pieces
* for JavaScript Promises:
    - primitives: Promise.resolve(), Promise.reject()
    - combinators: Promise.all(), Promise.race(), Promise.any(), Promise.allSettled()
        * for each of these combinators, the input is an iterable object containing 0+ Promises
            - so like an array of Promises
        * the output is 1 Promise

### Promise.all()

* returns a Promise which is:
    - Fulfilled if all promises are fulfilled
        * then its fulfillment value is an Array with the fulfillment values of promises
    - Rejected if at least one Promise is rejected
        * its rejection value is the rejection value of that Promise

In [1]:
// all promises are fulfilled so the array contains all fulfilled values
var promises = [
    Promise.resolve('result a'),
    Promise.resolve('result b'),
    Promise.resolve('result c')
];

Promise.all(promises)
    .then(arr => {
        arr.forEach(val => console.log(val))
    })

Promise { <pending> }

result a
result b
result c


In [3]:
// one promise is rejected
// so its rejection value is displayed in the .catch() callback
var promises = [
    Promise.resolve('result a'),
    Promise.resolve('result b'),
    Promise.reject('ERROR')
];

Promise.all(promises)
    .catch(err => {
        console.error(err)
    })

Promise { <pending> }

ERROR


#### Asynchronous .map() via Promise.all()

* higher order methods like .map() and .filter() are made for synchronous computations
* but if we passed in a Promise-based function into them, they would return an Array of Promises
    - normal code can't work with them
* but using Promise.all(), we can convert that Array of Promises into an array of normal values

In [4]:
function timesTwoAsync(x) {
    return new Promise(resolve => resolve(x * 2));
}

var arr = [1, 2, 3];
var promiseArr = arr.map(timesTwoAsync);
Promise.all(promiseArr)
    .then(result => {
        result.forEach(val => console.log(val));
    })

Promise { <pending> }

2
4
6


#### A more realistic .map() example

* we'll use .map() and Promise.all() to download text files from the web
* downloadText() uses the Promise-based fetch API to download a text file as a string
    - first it asynchronously retrieves a response (line A)
    - response.ok (line B) checks if there were errors such as "file not found"
    - if there weren't any, we use .text() (line C) to retrieve the contents of the file as a string

In [ ]:
function downloadText(url) {
    return fetch(url)
        .then(response => { // A
            if (!response.ok) { // B
                throw new Error(response.statusText);
            }
            return response.text(); // C
    });
}

var urls = [
    'http://example.com/first.txt',
    'http://example.com/second.txt',    
];

var promises = urls.map(url => downloadText(url));


// should print out ['First!', 'Second!']
Promise.all(promises)
    .then(arr => {
        arr.forEach(val => console.log(val))
})

#### A simple implementation of Promise.all()

* line A and line B are where the result Promise is settled
* after one of them is settled, the other can't change the settlement value b/c a Promise can only be settled once

In [7]:
// performs no safety checks

function all(iterable) {
    return new Promise((resolve, reject) => {
        let elementCount = 0;
        let result;
        
        let index = 0;
        for (const promise of iterable) {
            // Preserve the current value of the index
            const currentIndex = index;
            
            // Promise.then() takes in 2 arguments
            // a callback for the fulfilled case
            // and a callback for the rejected case
            promise.then(
                (value) => {
                    result[currentIndex] = value;
                    elementCount++;
                    // since we initialize result after all this code
                    // result.length === total number of items in the iterable
                    if (elementCount === result.length) {
                        resolve(result); // A
                    }
                },
                (err) => {
                    reject(err); // B
                }
            );
            index++;
        }
        
        // if the iterable is empty
        // just return an empty array
        if (index === 0) {
            resolve([]);
            return;
        }
        // Now we know how many Promises there are in 'iterable'.
        // we can wait until now with initializing result b/c
        // the callbacks of .then() are executed asynchronously
        result = new Array(index);
    });
}

### Promise.race()

* returns a Promise, q, which is settled as soon as the first Promise, p, among promises is settled
    - q has the same settlement value as p
* basically, whichever Promise is settled first among ALL of the promises is the one that decides what the settlement value is!

In [10]:
// settlement of the fulfilled Promise (line A) happens before the settlement of the rejected Promise (line B)
// so the result is also fulfilled (line C)

var promises = [
    new Promise((resolve, reject) => 
        setTimeout(() => resolve('result'), 100)), // A
    new Promise((resolve, reject) => 
        setTimeout(() => reject('ERROR'), 200)) // B
]

Promise.race(promises)
    .then(result => {
        console.log(result);
    })

Promise { <pending> }

result


In [14]:
// the rejection happens first
var assert = require('assert');

var promises = [
    new Promise((resolve, reject) => 
        setTimeout(() => resolve('result'), 200)), // A
    new Promise((resolve, reject) => 
        setTimeout(() => reject('ERROR'), 100)) // B
]

Promise.race(promises)
    .then(
        (result) => assert.fail(),
        (err) => console.error({err})
    )

Promise { <pending> }

{ err: 'ERROR' }
